In [ ]:
!pip install trax 

import ast
import string
import textwrap
import itertools
import numpy as np

import trax 
from trax import layers as tl
from trax.supervised import decoding

wrapper = textwrap.TextWrapper(width=70)
np.random.seed(2021)

     |████████████████████████████████| 522kB 6.7MB/s 
     |████████████████████████████████| 235kB 10.7MB/s 
     |████████████████████████████████| 3.4MB 9.6MB/s 
     |████████████████████████████████| 368kB 40.1MB/s 
     |████████████████████████████████| 1.2MB 38.2MB/s 
     |████████████████████████████████| 3.9MB 37.8MB/s 
     |████████████████████████████████| 2.1MB 35.9MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 3.3MB 34.3MB/s 
     |████████████████████████████████| 901kB 36.7MB/s 


In [1]:
examples = list(map(ast.literal_eval, open('data.txt', encoding='utf-8')))

for i in range(3):
    print(f'Example {i+1}: \n\n{examples[i]} \n')

Example 1: 

{'text': 'Этот текст подготовила группа аутичных взрослых, которые считают, что аналогичные статьи для родителей дают неточное представление об аутизме, в частности, они умалчивают о взрослых людях с расстройствами аутичного спектра. Аутизм является пожизненным состоянием, но по описаниям в СМИ создается ощущение, что он влияет только на маленьких детей. Аутичные дети вырастают и становятся аутичными взрослыми со своими специфическими потребностями, они также нуждаются в поддержке и имеют собственное мнение.'} 

Example 2: 

{'text': 'Я аутист, а не просто взрослый человек с аутизмом. Это часть того, кем я являюсь. Аутизм – это часть того, кто я есть. Я таким родился. Я бы не хотел это изменить. Можно признавать, что аутизм – это часть меня, а также уважать меня как человека со своими мыслями, чувствами и талантами. Я такой же человек, как и все остальные, и я заслуживаю такого же достоинства и уважения, как и другие люди.'} 

Example 3: 

{'text': 'Пожалуйста, употребляйт

In [ ]:
type(example_jsons[0].get('text'))

str

In [ ]:
texts = [example['text'] for example in examples]
texts[0]

'Этот текст подготовила группа аутичных взрослых, которые считают, что аналогичные статьи для родителей дают неточное представление об аутизме, в частности, они умалчивают о взрослых людях с расстройствами аутичного спектра. Аутизм является пожизненным состоянием, но по описаниям в СМИ создается ощущение, что он влияет только на маленьких детей. Аутичные дети вырастают и становятся аутичными взрослыми со своими специфическими потребностями, они также нуждаются в поддержке и имеют собственное мнение.'

In [ ]:
PAD, EOS, UNK = 0, 1, 2

def detokenize(np_array):
    return trax.data.detokenize(
        np_array,
        vocab_type='sentencepiece',
        vocab_file='sentencepiece.model',
        vocab_dir='.')

def tokenize(s):
  # operates on streams
    return next(trax.data.tokenize(
        iter([s]),
        vocab_type='sentencepiece',
        vocab_file='sentencepiece.model',
        vocab_dir='.'))
    
vocab_size = trax.data.vocab_size(
    vocab_type='sentencepiece',
    vocab_file='sentencepiece.model',
    vocab_dir='.')

tokenized = [(tokenize(word).tolist(), word) for word in texts[0].split()]
print(tokenized, '\n')

[([3, 2, 9592, 6725], 'Этот'), ([3, 14982, 6652, 10458], 'текст'), ([3, 2, 17238, 2, 2044, 9592, 25610, 17148], 'подготовила'), ([3, 2, 23912, 2, 2533], 'группа'), ([3, 2533, 3700, 18352, 2, 7184, 2], 'аутичных'), ([8724, 2, 10338, 5345, 6588, 2, 6], 'взрослых,'), ([3, 12377, 9592, 8452, 2, 1757], 'которые'), ([12681, 2, 2795, 15517, 2, 6725, 6], 'считают,'), ([3, 2, 9592], 'что'), ([3, 2533, 8194, 21044, 2, 2795, 2, 7184, 2, 1757], 'аналогичные'), ([3, 10458, 22581, 2, 2795], 'статьи'), ([3, 5814, 6588, 2], 'для'), ([3, 10338, 5814, 2795, 14982, 19229, 2], 'родителей'), ([3, 20000, 2, 6725], 'дают'), ([3, 14142, 9592, 2, 9890, 1757], 'неточное'), ([3, 2, 30051, 10458, 2533, 6609, 6588, 21302, 1757], 'представление'), ([3, 2044, 2], 'об'), ([3, 2533, 3700, 18352, 2, 25157, 6], 'аутизме,'), ([8724], 'в'), ([3, 2, 2533, 10458, 31585, 2795, 6], 'частности,'), ([3, 2044, 14391], 'они'), ([3, 3700, 20447, 6588, 2, 2795, 16624, 2, 6725], 'умалчивают'), ([3, 2044], 'о'), ([8724, 2, 10338, 534

In [ ]:
print(f"tokenized: {tokenize('текст')}\ndetokenized: {detokenize(tokenize('текст'))}")

tokenized: [    3 14982  6652 10458]
detokenized: текст


In [ ]:
input_str = texts[0]
print(f"input string:\n\n{input_str}\n")
inps, targs = tokenize_and_mask(input_str, randomizer=testing_rnd())
print(f"tokenized inputs:\n\n{inps}\n")
print(f"targets:\n\n{targs}")

input string:

Этот текст подготовила группа аутичных взрослых, которые считают, что аналогичные статьи для родителей дают неточное представление об аутизме, в частности, они умалчивают о взрослых людях с расстройствами аутичного спектра. Аутизм является пожизненным состоянием, но по описаниям в СМИ создается ощущение, что он влияет только на маленьких детей. Аутичные дети вырастают и становятся аутичными взрослыми со своими специфическими потребностями, они также нуждаются в поддержке и имеют собственное мнение.

tokenized inputs:

[31999, 9592, 6725, 3, 14982, 6652, 10458, 3, 2, 31998, 2044, 9592, 25610, 17148, 3, 2, 23912, 2, 31997, 2533, 3700, 18352, 2, 7184, 2, 8724, 2, 31996, 6588, 2, 6, 3, 12377, 9592, 8452, 2, 31995, 2, 2795, 15517, 2, 6725, 6, 3, 2, 31994, 2533, 8194, 21044, 2, 2795, 2, 7184, 2, 31993, 10458, 22581, 2, 2795, 3, 5814, 6588, 2, 31992, 5814, 2795, 14982, 19229, 2, 3, 20000, 2, 31991, 14142, 9592, 2, 9890, 1757, 3, 2, 30051, 31990, 6609, 6588, 21302, 1757, 3, 2044

In [ ]:
print('Inputs: \n\n', pretty_decode(inps))
print('\nTargets: \n\n', pretty_decode(targs))

Inputs: 

 <Z>тот текст  ⁇  <Y>отовила  ⁇ ру ⁇  <X>аути ⁇ н ⁇  в ⁇  <W>л ⁇ , котор ⁇  <V> ⁇ ита ⁇ т,  ⁇  <U>анало ⁇ и ⁇ н ⁇  <T>стат ⁇ и дл ⁇ <S>дителе ⁇  да ⁇ <R>нето ⁇ ное  ⁇ ред <Q>вление о ⁇  <P>ти ⁇ ме, в  ⁇ а <O>и, они ума<N>ива ⁇ т о в ⁇  <M>л ⁇  л ⁇ д ⁇  с <L>сстро ⁇ ствами  <K>ти ⁇ но ⁇ о с ⁇ е <J>ра.  ⁇ ути ⁇ м <I>вл ⁇ етс ⁇   ⁇ <H>и ⁇ ненн ⁇ м сост<G>нием, но  ⁇ о <F> ⁇ исани ⁇ м в  ⁇ <E>даетс ⁇  о ⁇ у<D>е,  ⁇ то он в<C>ет тол ⁇ ко на <B>лен ⁇ ки ⁇  дете <A>  ⁇ ути ⁇ н ⁇ е <z>ти в ⁇ раста ⁇ т <y> станов ⁇ тс ⁇ <x>ути ⁇ н ⁇ ми в ⁇ <w>л ⁇ ми со своими<v>е ⁇ и ⁇ и ⁇ ески<u> ⁇ отре ⁇ ност ⁇ ми, <t>ни так ⁇ е н <s>да ⁇ тс ⁇  в  ⁇ <r>ер ⁇ ке и и<q>т со ⁇ ственное <p>ение.

Targets: 

 <Z>  ⁇  <Y>од ⁇  <X>а  <W>рос <V>е с <U>то  <T>е <S> ро<R>т  <Q>ста<P>ау <O>стност<N>л ⁇  <M>рос <L> ра <K>ау <J>кт <I>  ⁇ <H>о ⁇ <G>о ⁇  <F> о<E> со ⁇ <D> ⁇ ени<C>ли ⁇  <B> ма <A> ⁇ . <z> де <y> и<x> а<w>рос<v> с ⁇ <u>ми  <t> о <s>у ⁇ <r>одд<q>ме ⁇ <p>мн


In [ ]:
inputs_targets_pairs = [tokenize_and_mask(text) for text in natural_language_texts]

def display_input_target_pairs(inputs_targets_pairs):
    for i, inp_tgt_pair in enumerate(inputs_targets_pairs, 1):
        inps, tgts = inp_tgt_pair
        inps, tgts = pretty_decode(inps), pretty_decode(tgts)
        print(f'[{i}]\n\n'
              f'inputs:\n{wrapper.fill(text=inps)}\n\n'
              f'targets:\n{wrapper.fill(text=tgts)}\n\n\n\n')
        
display_input_target_pairs(inputs_targets_pairs)

[1]

inputs:
 ⁇  <Z>т  <Y>кст  ⁇  <X>ото <W>ла  ⁇ ру ⁇ а аути ⁇ н ⁇  в <V>росл ⁇ ,
котор ⁇ е <U> ⁇ ита ⁇ т,  ⁇ то анало ⁇ и ⁇ н ⁇  <T> стат ⁇ и<S>дл ⁇
<R>родителе ⁇  да ⁇ т нето ⁇ ное  ⁇ редставление о ⁇   <Q>ути<P>ме, в
⁇  <O>стности, они у<N>л ⁇ ива ⁇  <M> о в ⁇ росл ⁇  л ⁇ д ⁇  <L>
<K>сстро ⁇ ствами а <J>ти ⁇ но ⁇ о <I> ⁇ <H>ктра. <G>ути ⁇  <F>  ⁇ вл
⁇ етс ⁇   ⁇ о ⁇ и ⁇ ненн ⁇ м состо ⁇ <E>ем, но<D> ⁇ о<C>о ⁇ исани ⁇ м
в  ⁇  <B> ⁇  <A>етс ⁇  о ⁇ у ⁇ ение,  ⁇ то он вли ⁇ ет тол ⁇ ко на
мален ⁇ ки ⁇  дете ⁇ .  <z>у <y> ⁇ <x> ⁇ е де<w> ⁇ раста ⁇ <v>и
<u>анов ⁇ тс ⁇  ау <t>н ⁇ ми <s> ⁇ росл ⁇ ми со своими с ⁇ е ⁇ и ⁇
и<r>ески<q>  ⁇ отре ⁇ ност ⁇ ми, они <p>к ⁇ е  <o>у ⁇  <n> ⁇ тс ⁇  <m>
⁇  <l>дер ⁇ ке и и<k> ⁇ т со ⁇ ственное м <j>ение.

targets:
<Z>то <Y>те <X>од ⁇  <W>ви <V> ⁇  <U> с <T>е<S> <R>  <Q>а<P> ⁇
<O>а<N>ма <M>т <L> с <K>ра <J>у <I> с<H>е<G> ⁇  <F>м<E>ни<D> <C>  <B>
со <A>да <z> ⁇  <y>ти<x>н<w>ти в<v>т <u>ст <t>ти ⁇  <s> в<r> ⁇
<q>ми<p>та <o>н <n>да <m> в <l>од<k>ме <j>н




